In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
insurance=spark.read.csv("/home/phani/Desktop/Insurance_Data.csv",header=True,inferSchema=True)


In [3]:
insurance.show(5)


+-------------+------------------+-----------+------------+-------------+-----------+---------------+-----------+
|Policy Number|Number of Vehicles|Average Age|Gender Dummy|Married Dummy|Avg Veh Age|Fuel Type Dummy|     Losses|
+-------------+------------------+-----------+------------+-------------+-----------+---------------+-----------+
|       150023|                 1|       20.6|           1|            0|      12.97|              0|203.3195946|
|       152720|                 1|       42.4|           0|            1|       8.02|              0|402.1973024|
|       174207|                 1|       42.4|           1|            0|        2.5|              1|553.1320959|
|       139895|                 1|       42.4|           0|            0|      12.97|              0|292.4305792|
|       174407|                 1|       20.6|           1|            1|      12.97|              0|246.5405761|
+-------------+------------------+-----------+------------+-------------+-----------+---

In [4]:
from pyspark.ml.linalg import Vectors


In [5]:
insurancedf=insurance.rdd.map(lambda x:[Vectors.dense(x[1:6]),x[-1]]).toDF(['features','label'])

In [6]:
insurancedf.show(5)

+--------------------+-----------+
|            features|      label|
+--------------------+-----------+
|[1.0,20.6,1.0,0.0...|203.3195946|
|[1.0,42.4,0.0,1.0...|402.1973024|
|[1.0,42.4,1.0,0.0...|553.1320959|
|[1.0,42.4,0.0,0.0...|292.4305792|
|[1.0,20.6,1.0,1.0...|246.5405761|
+--------------------+-----------+
only showing top 5 rows



In [7]:
from pyspark.ml.regression import LinearRegression


In [9]:
lr=LinearRegression(featuresCol='features',labelCol='label') 

In [10]:
lr_model=lr.fit(insurancedf)

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

In [12]:
pred=lr_model.transform(insurancedf)

In [13]:
pred.show(5)

+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[1.0,20.6,1.0,0.0...|203.3195946| 447.3748442510993|
|[1.0,42.4,0.0,1.0...|402.1973024|433.68708068775413|
|[1.0,42.4,1.0,0.0...|553.1320959| 549.7032644125243|
|[1.0,42.4,0.0,0.0...|292.4305792| 156.4950203379566|
|[1.0,20.6,1.0,1.0...|246.5405761| 582.8308907793871|
+--------------------+-----------+------------------+
only showing top 5 rows



In [14]:
evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='label',metricName="rmse")

In [15]:
rmse=evaluator.evaluate(pred)

In [16]:
print(rmse)

174.68105816233438


In [17]:
lr_model.coefficients

DenseVector([-2.1789, -9.058, 93.4151, 135.456, -28.6335])

In [18]:
lr_model.coefficients

DenseVector([-2.1789, -9.058, 93.4151, 135.456, -28.6335])

In [19]:
lr_model.summary.meanSquaredError

30513.47208071285

In [20]:
lr_model.intercept


914.1106930643006

In [21]:
lr_model.summary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-244.05524965109933|
| -31.48977828775412|
| 3.4288314874756907|
| 135.93555886204342|
| -336.2903146793871|
|-6.9697390479316255|
|-129.58535505234676|
| 218.05595337214336|
| -409.6493324207121|
|-246.64321499092185|
|  180.8989659206129|
| 115.43830733375569|
|-192.38022346263412|
|  56.85957936216832|
|-33.124428847931654|
|  64.75197717357815|
| 126.99027215539036|
| 189.81392952061287|
| 113.88503796928791|
| -68.87191770242441|
+-------------------+
only showing top 20 rows



In [22]:
pred.show(10)  ## above trust val is below label-prediction value same goes for all res 9 values

+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[1.0,20.6,1.0,0.0...|203.3195946| 447.3748442510993|
|[1.0,42.4,0.0,1.0...|402.1973024|433.68708068775413|
|[1.0,42.4,1.0,0.0...|553.1320959| 549.7032644125243|
|[1.0,42.4,0.0,0.0...|292.4305792| 156.4950203379566|
|[1.0,20.6,1.0,1.0...|246.5405761| 582.8308907793871|
|[1.0,42.4,1.0,0.0...|384.6763948|391.64613384793165|
|[1.0,42.4,0.0,1.0...|462.1588562| 591.7442112523468|
|[1.0,65.1,0.0,0.0...|168.9340361|-49.12191727214338|
|[1.0,65.1,1.0,1.0...|69.89304091| 479.5423733307121|
|[1.0,20.6,0.0,1.0...|242.7725761|489.41579109092186|
+--------------------+-----------+------------------+
only showing top 10 rows



In [23]:
lr_model.summary.objectiveHistory

[0.0]

In [24]:
lr_model.summary.pValues

[0.1414190209338151, 0.0, 0.0, 0.0, 0.0, 0.0]

In [25]:
lr_model.summary.tValues  ## T statistic values with degrees of freedom - Test Statistics

[-1.4706032847815143,
 -104.40700440852557,
 33.05251492798508,
 47.921350426590024,
 -79.4179853381474,
 131.02426378545263]

In [26]:
lr_model.summary.totalIterations

1

In [27]:
insurancedf.count()

15290

In [28]:
traindata,testdata=insurancedf.randomSplit([0.70,0.30],seed=123) ## spliting train and test data and seed is the random generator##

In [29]:
traindata.count()

10659

In [30]:
testdata.count()

4631

In [31]:
lrtrain=LinearRegression(featuresCol='features',labelCol='label') 

In [32]:
lr_train=lr.fit(traindata)

In [33]:
pred_train=lr_train.transform(traindata)

In [34]:
pred_train.show(5)

+--------------------+-----------+-----------------+
|            features|      label|       prediction|
+--------------------+-----------+-----------------+
|[1.0,20.6,0.0,0.0...|342.5996055|653.8026127805778|
|[1.0,20.6,0.0,0.0...| 399.165851|653.8026127805778|
|[1.0,20.6,0.0,0.0...|458.1587158|653.8026127805778|
|[1.0,20.6,0.0,0.0...| 459.488058|653.8026127805778|
|[1.0,20.6,0.0,0.0...|481.9361346|653.8026127805778|
+--------------------+-----------+-----------------+
only showing top 5 rows



In [35]:
evaluator.evaluate(pred_train)

174.06488297135078

In [36]:
evaluator2=RegressionEvaluator(predictionCol='prediction',labelCol='label',metricName="r2")

In [37]:
evaluator2.evaluate(pred_train)

0.5346702795957955

In [38]:
lrtest=LinearRegression(featuresCol='features',labelCol='label') 

In [39]:
lr_test=lr.fit(testdata)

In [40]:
pred_test=lr_test.transform(testdata)

In [41]:
pred_test.show()

+--------------------+-----------+-----------------+
|            features|      label|       prediction|
+--------------------+-----------+-----------------+
|[1.0,20.6,0.0,0.0...|345.0465518|653.5643205998598|
|[1.0,20.6,0.0,0.0...|346.1635562|653.5643205998598|
|[1.0,20.6,0.0,0.0...|378.5785956|653.5643205998598|
|[1.0,20.6,0.0,0.0...|488.2883074|653.5643205998598|
|[1.0,20.6,0.0,0.0...|503.0228692|653.5643205998598|
|[1.0,20.6,0.0,0.0...|805.5978863|653.5643205998598|
|[1.0,20.6,0.0,0.0...|806.8782918|653.5643205998598|
|[1.0,20.6,0.0,0.0...|837.3677817|653.5643205998598|
|[1.0,20.6,0.0,0.0...|282.2230386| 497.951516790611|
|[1.0,20.6,0.0,0.0...|299.9629532| 497.951516790611|
|[1.0,20.6,0.0,0.0...|307.6119306| 497.951516790611|
|[1.0,20.6,0.0,0.0...|335.9492312| 497.951516790611|
|[1.0,20.6,0.0,0.0...|343.4349944| 497.951516790611|
|[1.0,20.6,0.0,0.0...|363.1691719| 497.951516790611|
|[1.0,20.6,0.0,0.0...|370.6644896| 497.951516790611|
|[1.0,20.6,0.0,0.0...|382.4581608| 497.9515167

In [42]:
evaluator3=RegressionEvaluator(predictionCol='prediction',labelCol='label',metricName="r2")

In [43]:
evaluator3.evaluate(pred_test)

0.5055331292063

In [44]:
from pyspark.ml.regression import DecisionTreeRegressor  ## Decision Tree

In [45]:
dtmodel=DecisionTreeRegressor(featuresCol='features')

In [46]:
dt_model=dtmodel.fit(insurancedf)

In [47]:
dtpred=dt_model.transform(insurancedf)

In [48]:
dtpred.show(5)

+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[1.0,20.6,1.0,0.0...|203.3195946| 333.2638523903444|
|[1.0,42.4,0.0,1.0...|402.1973024|433.49622672352524|
|[1.0,42.4,1.0,0.0...|553.1320959| 547.5435941172063|
|[1.0,42.4,0.0,0.0...|292.4305792|249.49646718962936|
|[1.0,20.6,1.0,1.0...|246.5405761| 742.3726205902141|
+--------------------+-----------+------------------+
only showing top 5 rows



In [49]:
evaluator.evaluate(dtpred)

157.9959675570192

In [50]:
from pyspark.ml.regression import RandomForestRegressor  ## importing random forest

In [51]:
dtforest=RandomForestRegressor(featuresCol='features')

In [52]:
dt_forest=dtforest.fit(insurancedf)

In [53]:
dtpred_for=dt_forest.transform(insurancedf)

In [54]:
dtpred_for.show(5)


+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[1.0,20.6,1.0,0.0...|203.3195946|394.52831527148516|
|[1.0,42.4,0.0,1.0...|402.1973024| 412.6923644670822|
|[1.0,42.4,1.0,0.0...|553.1320959| 544.2492924596918|
|[1.0,42.4,0.0,0.0...|292.4305792|294.59234701428454|
|[1.0,20.6,1.0,1.0...|246.5405761| 644.2949310397112|
+--------------------+-----------+------------------+
only showing top 5 rows



In [55]:
evaluator.evaluate(dtpred_for)

165.45466792558872